Pattern 1: Reprocessing Window (Simplest & Common)

Always reprocess last N days so late data automatically gets picked up.
Use when
- Late data is bounded (e.g. max 2–7 days)
- Volume is manageable
- Aggregates can be recomputed cheaply

Tradeoff
- Rewrites partitions every run
- Not ideal if data is huge

In [0]:
from pyspark.sql.functions import current_date, date_sub

# Reprocess last 3 days
reprocess_days = 3

df = spark.read.format("delta").load("/bronze/events")

filtered_df = df.filter(
    df.event_date >= date_sub(current_date(), reprocess_days)
)

filtered_df.write.format("delta") \
    .mode("overwrite") \
    .option("replaceWhere", f"event_date >= date_sub(current_date(), {reprocess_days})") \
    .save("/silver/events")


Pattern 2: Event-Time Partitioning + Append (Bronze)

Late data is written to the correct partition, not “today’s” partition.

Use when
- You want immutability
- Late data is frequent
- You don’t want special handling logic

**Key point** 
Late data for Jan 22 arriving on Jan 24 still lands in event_date=2024-01-22

In [0]:
df = spark.read.json("/landing/events")

df = df.withColumn("event_date", df.event_timestamp.cast("date"))

df.write.format("delta") \
    .mode("append") \
    .partitionBy("event_date") \
    .save("/bronze/events")


Pattern 3: MERGE (Upsert) for Late & Corrected Data (Silver)

Late data updates or inserts records instead of duplicating them.

Use when
- Data has natural keys
- Updates are expected
- You need correctness over time

Pros:
- Idempotent
- Safe re-runs
- Late data naturally handled

from delta.tables import DeltaTable

silver = DeltaTable.forPath(spark, "/silver/events")

updates = spark.read.format("delta").load("/bronze/events")

silver.alias("t").merge(
    updates.alias("s"),
    "t.event_id = s.event_id"
).whenMatchedUpdateAll() \
 .whenNotMatchedInsertAll() \
 .execute()


Pattern 4: Separate Late Data Detection (Explicit Handling)

Detect late data and process it explicitly, not implicitly.

Use when
- SLA-driven systems
- Audits required
- Late data is an exception, not the norm

Risk
- Extra complexity
- Must merge downstream

In [0]:
from pyspark.sql.functions import current_timestamp

late_threshold_hours = 24

df = spark.read.json("/landing/events")

on_time = df.filter(
    df.ingest_time >= df.event_time
)

late = df.filter(
    df.ingest_time < df.event_time
)

on_time.write.format("delta").mode("append").save("/bronze/on_time")
late.write.format("delta").mode("append").save("/bronze/late_events")


In [0]:
# Controlled late processing
late_events = spark.read.format("delta").load("/bronze/late_events")


Pattern 5: Watermark + Backfill Trigger (ADF Friendly)

Process normally using watermark, trigger backfill when late data detected.

Use when
- Batch systems
- ADF-based orchestration
- Business-approved reprocessing



In [0]:
last_watermark = "2024-01-24 10:00:00"

df = spark.read.format("delta").load("/bronze/events")

incremental = df.filter(df.updated_at > last_watermark)


In [0]:
late_data = df.filter(df.updated_at <= last_watermark)

if late_data.count() > 0:
    # Trigger backfill pipeline via ADF
    pass


Pattern 6: Gold Layer – Correcting Aggregates (MOST MISSED)

Never patch aggregates. Recompute affected partitions only.

Use when
- Metrics matter
- Late data impacts KPIs

In [0]:
affected_dates = (
    late_data
    .select("event_date")
    .distinct()
    .collect()
)

for row in affected_dates:
    date = row["event_date"]

    base = spark.read.format("delta").load("/silver/events") \
        .filter(f"event_date = '{date}'")

    agg = base.groupBy("product_id") \
        .count()

    agg.write.format("delta") \
        .mode("overwrite") \
        .option("replaceWhere", f"event_date = '{date}'") \
        .save("/gold/daily_sales")


When to Use Which (Summary Table)

Pattern -	Best For
- Reprocessing window	- Small bounded lateness
- Event time partitioning -	Immutable ingestion
- MERGE -	Updates + late data
- Explicit late stream -	SLA / audit systems
- Watermark + backfill	- ADF batch pipelines
- Targeted gold recompute -	KPI correctness

Late-arriving data is ingested using event-time partitioning, corrected via idempotent MERGE operations in silver, and aggregates are recomputed only for impacted partitions in gold